# Unit 12. 베스트 모델 저장하기
### - 이진 분류 - 유방암 예측 사례

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

- 데이터 준비

In [ ]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
cancer_std = scaler.fit_transform(cancer.data)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    cancer_std, cancer.target, stratify=cancer.target, test_size=0.2, random_state=2022
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

### 1. Best Model 저장하고 불러오기

- 모델 정의

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential([
    Dense(100, input_shape=(30,), activation='relu'),
    Dense(24, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.summary()

- 모델 설정

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

- Callback 설정 - 베스트 모델 저장

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

modelpath = 'best_model.h5'
mc = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1,
                     save_best_only=True)

- 모델 학습

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=50,
                    verbose=1, callbacks=[mc])

- 베스트 모델 불러오기

In [ ]:
from tensorflow.keras.models import load_model

best_model = load_model(modelpath)

- 모델 평가

In [ ]:
best_model.evaluate(X_test, y_test)

### 2. 베스트 모델 저장하고 조기 종료하기

- Dropout layer 추가

In [ ]:
from tensorflow.keras.layers import Dropout

model2 = Sequential()
model2.add(Dense(100, input_dim=30, activation='relu'))
model2.add(Dense(24, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))
model2.summary()

In [ ]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

- 조기 종료 조건 설정

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', patience=30)

In [ ]:
modelpath2 = 'best_model2.h5'
mc2 = ModelCheckpoint(filepath=modelpath2, monitor='val_loss', verbose=1,
                      save_best_only=True)

In [ ]:
history2 = model2.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=50,
                      verbose=1, callbacks=[mc2, es])

In [ ]:
best_model2 = load_model(modelpath2)
best_model2.evaluate(X_test, y_test)

In [ ]:
y_acc2 = history2.history['accuracy']
y_vloss2 = history2.history['val_loss']
xs2 = np.arange(1,len(y_acc2)+1)        # epoch

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(xs2, y_acc2, label='train accuracy')
plt.plot(xs2, y_vloss2, label='validation loss')
plt.legend()
plt.grid()
plt.ylim([0,1.1])
plt.xlabel('Epoch')
plt.title('Best model + Early stop')
plt.show()